In [ ]:
from IPython.display import display, HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

In [ ]:
%load_ext nb_mypy

# Breadth First Search

In [ ]:
from typing import TypeVar, Callable
State    = TypeVar('State')
NxtStFct = Callable[[State], set[State]] 

Given a `state` and a parent dictionary `Parent`, the function `path_to` returns a path leading from `start` to the given `state`.

In [ ]:
def path_to(state: State, Parent: dict[State, State]) -> list[State]:
    p = Parent[state]
    if p == state:
        return [state]
    return path_to(p, Parent) + [state]

---

The function `search` takes three arguments to solve a *search problem*:
- `start` is the *start state* of the search problem,
- `goal` is the *goal state*, and
- `next_states` is a function with signature $\texttt{next_states}:Q \rightarrow 2^Q$, where $Q$ is the set of states.
   For every state $s \in Q$, $\texttt{next_states}(s)$ is the set of states that can be reached from $s$ in one step.

If successful, `search` returns a path from `start` to `goal` that is a solution of the search problem
$$ \langle Q, \texttt{next_states}, \texttt{start}, \texttt{goal} \rangle. $$
The implementation of `search` uses the algorithm *breadth first search* to find a path from `start` to `goal`.

At the start of the $n^\textrm{th}$ iteration of the `while` loop, the following invariants are satisfied:
* `Frontier` contains exactly those states that have distance of $n-1$ from `start`.
* `Visited`  contains those states that have distance from start that is less than `n-1`.
* `Parent` is a dictionary. The keys of this dictionaries are all states from the sets `Visited`, `Frontier`, and `NewFrontier`.    Furthermore, the following invariant holds for all states $x \not= \texttt{start}$:
  - If $x = \texttt{Parent}[y]$, then $y \in \texttt{next_states}(x)$.  

In [ ]:
def search(start: State, goal: State, next_states: NxtStFct) -> list[State] | None:
    Frontier            = { start }
    Visited: set[State] = set()
    Parent              = { start: start }
    while Frontier:
        NewFrontier = set()
        for s in Frontier:
            for ns in next_states(s):
                if ns not in Visited and ns not in Frontier:
                    NewFrontier.add(ns)
                    Parent[ns] = s
                    if ns == goal:
                        print("number of states: ", len(Visited) + len(Frontier) + len(NewFrontier))
                        return path_to(goal, Parent)
        Visited |= Frontier
        Frontier = NewFrontier
        print(len(Frontier))
    return None

# Display Code

In [ ]:
import graphviz as gv

The function $\texttt{toDot}(\texttt{source}, \texttt{Edges}, \texttt{Fringe}, \texttt{Visited})$ takes a graph that is represented by 
its `Edges`, a set of nodes `Fringe`, and set `Visited` of nodes that have already been visited.

In [ ]:
def toDot(source, goal, Edges, Frontier, Visited, Parent=None):
    V = set()
    for x, L in Edges.items():
        V.add(x)
        for y in L:
            V.add(y)
    dot = gv.Digraph(node_attr={'shape': 'record', 'style': 'rounded'})
    dot.attr(rankdir='LR')
    dot.attr(size="12,20") 
    for x in V:
        if x == source:
            dot.node(str(x), color='blue', shape='doublecircle')
        elif x in Frontier and x == goal:
            dot.node(str(x), label=str(x), color='magenta')
        elif x in Frontier:
            dot.node(str(x), label=str(x), color='red')
        elif x in Visited:
            dot.node(str(x), label=str(x), color='blue')
        else:
            dot.node(str(x), label=str(x))
    if Parent:        
        Path = path_to(goal, Parent)
    for u in V:
        if Edges.get(u):
            for v in Edges[u]:
                if Parent and v in Path and Parent[v] == u:
                    dot.edge(str(u), str(v), color='brown', style='bold')                    
                else:
                    dot.edge(str(u), str(v))
    return dot

# Testing

In [ ]:
def next_states_test(node):
    x, y = node
    return { (x+1, y), (x, y+1) }

In [ ]:
def create_edges(n):
    Edges = {}
    for row in range(n):
        for col in range(n):
            if (row, col) != (n-1, n-1):
                Edges[(row, col)] = list(next_states_test((row, col)))
    for k in range(n-1):
        Edges[(k, n-1)] = [(k+1, n-1)]
        Edges[(n-1, k)] = [(n-1, k+1)]
    return Edges

In [ ]:
def search_show(start, goal, next_states, Edges):
    Visited  = set()
    Frontier = { start }
    Parent   = { start: start }
    while len(Frontier) > 0:
        display(toDot(start, goal, Edges, Frontier, Visited))
        NewFrontier = set()
        Visited    |= Frontier
        for s in Frontier:
            for ns in next_states(s):
                if not (ns in Visited):
                    NewFrontier.add(ns)
                    Parent[ns] = s
                    if ns == goal:
                        display(toDot(start, goal, Edges, NewFrontier, Visited, Parent))
                        return 
        Frontier = NewFrontier

In [ ]:
def main(n):
    Edges = create_edges(n)
    search_show((0,0), (n-1, n -1), next_states_test, Edges)

In [ ]:
main(5)

# Saving the Infidels

In [ ]:
%run 01-Missionaries.ipynb

Unfortunately, the command `%run` does not cooperate with the extension `nb_mypy`.  Therefore, we have to unload this extension. 

In [ ]:
%unload_ext nb_mypy

In [ ]:
dot_graph(createRelation(start))

In [ ]:
%%time
Path = search(start, goal, next_states)
printPath(Path)

# Solving the Three Thieves Puzzle

The code below solves the *Three Thieves Puzzle*.   In order to solve this puzzle, you have to add some code to the notebook `Three-Thieves.ipynb`.

In [ ]:
%run Three-Thieves-Solution.ipynb

In [ ]:
%unload_ext nb_mypy

In [ ]:
dot_graph(createRelation(start))

In [ ]:
%%time
Path = search(start, goal, next_states)
printPath(Path)

# Solving the Sliding Puzzle

In [ ]:
%run 03-Sliding-Puzzle.ipynb

In [ ]:
!pip install memory_profiler

In [ ]:
%load_ext memory_profiler

In [ ]:
%%time
%memit Path = search(start, goal, next_states)

In [ ]:
animation(Path) # type: ignore